In [48]:
#import sys
#!{sys.executable} -m pip install scipy

import yfinance as yf
import pandas as pd
import numpy as np
from ta.momentum import StochRSIIndicator
from ta.momentum import RSIIndicator
from ta.volume import ForceIndexIndicator
from ta.trend import SMAIndicator
from ta.volatility import BollingerBands
from ta.volatility import KeltnerChannel
#from scipy.stats import percentileofscore
payload=pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[0]
#print(first_table.head())
sip=np.array(payload.iloc[:,0])
sip=sip[sip!='BRK.B']
sip=sip[sip!='BF.B']
df=pd.DataFrame()
dfout=pd.DataFrame()
start_date="2000-10-01"
end_date="2021-10-01"
txt = "{:.2%}"
wind=14
for i in range(0,200):
    stk=sip[i]
    #df = yf.download(stk, start_date, end_date,interval=intr).dropna()
    print(stk)
    dfd = yf.download(stk, start_date, end_date,interval='1d').dropna()
    dfd['Date'] = pd.to_datetime(dfd.index)
    dfd['Week_Number'] = dfd['Date'].dt.isocalendar().week
    dfd['Year'] = dfd['Date'].dt.isocalendar().year
    dfd['Close_1']=dfd['Close'].shift(1)
    dfd['Volume_1']=dfd['Volume'].shift(1)
    dfd['Low1']=dfd['Low'].shift(1)
    dfd['High1']=dfd['High'].shift(1)
    dfd.set_index(['Year','Week_Number'],inplace=True)
    dfd['Mon']=dfd['Date'].dt.strftime('%m-%Y')

    #dfm=dfd.groupby(month('Date')).agg({'Date':'first','Open':'first', 'High':'max', 'Low':'min', 'Close':'last','Volume':'sum'})
    dfm=dfd.resample('M', on='Date').agg({'Date':'first','Open':'first', 'High':'max', 'Low':'min', 'Close':'last','Volume':'sum','Mon':'first'})
    #print(dfm.tail())
    #print(dfd.tail())
    dfd=dfd.dropna()
    dfw = dfd.groupby(['Year','Week_Number']).agg({'Date':'first','Open':'first', 'High':'max', 'Low':'min', 'Close':'last','Volume':'sum'})
    #dfm = dfd.groupby(['Mon']).agg({'Date':'first','Open':'first', 'High':'max', 'Low':'min', 'Close':'last','Volume':'sum'})
    #dfm.to_csv('dfm.csv')
    
    dfw['Close_1']=dfw['Close'].shift(1)
    dfw['Volume_1']=dfw['Volume'].shift(1)
    dfw['Low1']=dfw['Low'].shift(1)
    dfw['High1']=dfw['High'].shift(1)
    dfw["kw"]=StochRSIIndicator(close=dfw.Close_1,window=wind,smooth1=3, smooth2=3).stochrsi_k() 
    #dfw.set_index('Date',inplace=True)
    
    dfm['Close_1']=dfm['Close'].shift(1)
    dfm["km"]=StochRSIIndicator(close=dfm.Close_1,window=wind,smooth1=3, smooth2=3).stochrsi_k() 
    
    
    dfd=pd.merge(left=dfd,right=dfw['kw'],how='left',left_index=True,right_index=True,suffixes=[None,'w'])
    dfd.set_index('Mon',inplace=True)
    dfm.set_index('Mon',inplace=True)
    dfd=pd.merge(left=dfd,right=dfm['km'],how='left',left_on='Mon',right_on='Mon',suffixes=[None,'m'])
    
    
    #dfd=pd.merge(left=dfd,right=dfm['Close_1'],how='left',left_on='Mon',right_on='Mon',suffixes=[None,'m'])
    #print(dfd.tail())
    dfd['efid']=ForceIndexIndicator(close=dfd.Close_1,volume=dfd.Volume_1,window=wind,fillna=True).force_index()
    dfd['efi_rsid']=RSIIndicator(close=dfd.efid,window=wind,fillna=True).rsi()
    dfd["kd"]=StochRSIIndicator(close=dfd.Close_1,window=wind,smooth1=3, smooth2=3).stochrsi_k() 
    #dfd['bbh']=BollingerBands(close=dfd.Close_1,window=20,window_dev=2,fillna=True).bollinger_hband()
    #dfd['bbm']=BollingerBands(close=dfd.Close_1,window=20,window_dev=2,fillna=True).bollinger_mavg()
    #dfd['bbl']=BollingerBands(close=dfd.Close_1,window=20,window_dev=2,fillna=True).bollinger_lband()
    dfd['bbhc']=BollingerBands(close=dfd.Close_1,window=20,window_dev=2,fillna=True).bollinger_hband_indicator()
    dfd['bblc']=BollingerBands(close=dfd.Close_1,window=20,window_dev=2,fillna=True).bollinger_lband_indicator()
    dfd['kchc']=KeltnerChannel(close=dfd.Close_1,high=dfd.High1,low=dfd.Low1,window=20,fillna=True).keltner_channel_hband_indicator() 
    dfd['kclc']=KeltnerChannel(close=dfd.Close_1,high=dfd.High1,low=dfd.Low1,window=20,fillna=True).keltner_channel_lband_indicator()


    dfd['sma200']=SMAIndicator(close=dfd.Close_1,window=200,fillna=True).sma_indicator()
    dfd['sma100']=SMAIndicator(close=dfd.Close_1,window=100,fillna=True).sma_indicator()
    dfd['sma50']=SMAIndicator(close=dfd.Close_1,window=50,fillna=True).sma_indicator()
    dfd['sma20']=SMAIndicator(close=dfd.Close_1,window=20,fillna=True).sma_indicator()
    dfd['sma10']=SMAIndicator(close=dfd.Close_1,window=10,fillna=True).sma_indicator()
    dfd["Chng"]=(dfd.Close_1-dfd.Close)
    dfd['smachk200']=np.select([dfd.Close_1>dfd.sma200],[1],0)
    dfd['smachk100']=np.select([dfd.Close_1>dfd.sma100],[1],0)
    dfd['smachk50']=np.select([dfd.Close_1>dfd.sma50],[1],0)
    dfd['smachk20']=np.select([dfd.Close_1>dfd.sma20],[1],0)
    dfd['smachk10']=np.select([dfd.Close_1>dfd.sma10],[1],0)
    #dfd['bbchk']=np.select([dfd.Close_1>dfd.bbh],[1],0)
    dfd=dfd.dropna()
    #print(dfd.tail())
    #print(perc)
    dfd.to_csv('dfd.csv')
    dfd_90=dfd[dfd['Chng']>=np.nanpercentile(dfd.Chng,95)]
    sma200=(np.average(dfd_90.smachk200))-(np.average(dfd.smachk200))
    sma100=(np.average(dfd_90.smachk100))-(np.average(dfd.smachk100))
    sma50=(np.average(dfd_90.smachk50))-(np.average(dfd.smachk50))
    sma20=(np.average(dfd_90.smachk20))-(np.average(dfd.smachk20))
    sma10=(np.average(dfd_90.smachk10))-(np.average(dfd.smachk10))
    #print(np.average(dfd_90.bbchk))
    kd=((np.nanpercentile(dfd_90.kd,50))-(np.nanpercentile(dfd.kd,50)))
    kw=((np.nanpercentile(dfd_90.kw,50))-(np.nanpercentile(dfd.kw,50)))
    km=((np.nanpercentile(dfd_90.km,50))-(np.nanpercentile(dfd.km,50)))
    efi_rsi=((np.nanpercentile(dfd_90.efi_rsid,50))-(np.nanpercentile(dfd.efi_rsid,50)))/100
    bbhc=(np.average(dfd_90.bbhc))-(np.average(dfd.bbhc))
    bblc=(np.average(dfd_90.bblc))-(np.average(dfd.bblc))
    kchc=(np.average(dfd_90.kchc))-(np.average(dfd.kchc))
    kclc=(np.average(dfd_90.kclc))-(np.average(dfd.kclc))
    temp=pd.Series([stk,sma200,sma100,sma50,sma20,sma10,kd,kw,km,efi_rsi,bbhc,bblc,kchc,kclc])
    dfout=dfout.append(temp,ignore_index=True)

dfout=dfout.dropna()
print(dfout)
temp1=[]
for i in range(1,len(dfout.columns)): 
    out=(np.average(dfout.iloc[:,i]))
    temp1.append(out)
out2=pd.DataFrame(temp1)
out2=out2.set_index([pd.Index(['sma200','sma100','sma50','sma20','sma10','kd','km','kw','efi_rsi','bbhc','bblc','kchc','kclc'])])
#pd.options.display.float_format = '{:.2%}'.format
print((out2))


MMM
[*********************100%***********************]  1 of 1 completed
ABT
[*********************100%***********************]  1 of 1 completed
ABBV
[*********************100%***********************]  1 of 1 completed
ABMD
[*********************100%***********************]  1 of 1 completed
ACN
[*********************100%***********************]  1 of 1 completed
ATVI
[*********************100%***********************]  1 of 1 completed
ADBE
[*********************100%***********************]  1 of 1 completed
AMD
[*********************100%***********************]  1 of 1 completed
AAP
[*********************100%***********************]  1 of 1 completed
AES
[*********************100%***********************]  1 of 1 completed
AFL
[*********************100%***********************]  1 of 1 completed
A
[*********************100%***********************]  1 of 1 completed
APD
[*********************100%***********************]  1 of 1 completed
AKAM
[*********************100%******************

/srv/conda/envs/notebook/lib/python3.7/site-packages/numpy/lib/nanfunctions.py:1372: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
/srv/conda/envs/notebook/lib/python3.7/site-packages/numpy/lib/function_base.py:380: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis)
/srv/conda/envs/notebook/lib/python3.7/site-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


[*********************100%***********************]  1 of 1 completed
CAT
[*********************100%***********************]  1 of 1 completed
CBOE
[*********************100%***********************]  1 of 1 completed
CBRE
[*********************100%***********************]  1 of 1 completed
CDW
[*********************100%***********************]  1 of 1 completed
CE
[*********************100%***********************]  1 of 1 completed
CNC
[*********************100%***********************]  1 of 1 completed
CNP
[*********************100%***********************]  1 of 1 completed
CDAY
[*********************100%***********************]  1 of 1 completed
CERN
[*********************100%***********************]  1 of 1 completed
CF
[*********************100%***********************]  1 of 1 completed
CRL
[*********************100%***********************]  1 of 1 completed
SCHW
[*********************100%***********************]  1 of 1 completed
CHTR
[*********************100%*********************